[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/UMCF.ipynb)

# 사용자-사용자 메모리 기반 방법 협조 필터링(User-User Memory Based Collaborative Filtering)

In [1]:
# Colab용 notebook입니다. 이 notebook 한 장에서 여러 데이터의 다운로드부터, 추천까지 완결하도록 되어 있습니다(예측 평가는 미포함)
# MovieLens 데이터를 아직 다운로드 하지 않았다면, 이 셀을 실행해서 다운로드합니다.
# MovieLens 데이터 분석은 data_download.ipynb를 참조합니다.

# 데이터 다운로드와 압축 풀기
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

--2022-12-27 05:54:30--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘../data/ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  91.0MB/s    in 0.7s    

2022-12-27 05:54:31 (91.0 MB/s) - ‘../data/ml-10m.zip’ saved [65566137/65566137]

Archive:  ../data/ml-10m.zip
   creating: ../data/ml-10M100K/
  inflating: ../data/ml-10M100K/allbut.pl  
  inflating: ../data/ml-10M100K/movies.dat  
  inflating: ../data/ml-10M100K/ratings.dat  
  inflating: ../data/ml-10M100K/README.html  
  inflating: ../data/ml-10M100K/split_ratings.sh  
  inflating: ../data/ml-10M100K/tags.dat  


In [2]:
# Movielens 데이터 로딩(데이터량이 많으므로, 로딩에 시간이 걸릴 수 있습니다)
import pandas as pd

# movieID와 제목만 사용
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genre를 list 형식으로 저장한다
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# 사용자가 부여한 영화의 태그 정보를 로딩한다
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tag를 소문자로 바꾼다
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tag를 영화별로 list 형식으로 저장한다
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# 태그 정보를 결합한다
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 평갓값 데이터만 로딩한다
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# 데이터량이 많으므로 사용자수를 1000으로 줄여서 시험해본다
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 영화 데이터와 평가 데이터를 결합한다
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 학습용과 데이터용으로 데이터를 나눈다
# 각 사용자의 최근 5건의 영화를 평가용으로 사용하고, 나머지는 학습용으로 사용한다
# 우선, 각 사용자가 평가한 영화의 순서를 계산한다
# 최근 부여한 영화부터 순서를 부여한다(1에서 시작)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [3]:
import numpy as np
# 피어슨 상관 계수
def peason_coefficient(u: np.ndarray, v: np.ndarray) -> float:
    u_diff = u - np.mean(u)
    v_diff = v - np.mean(v)
    numerator = np.dot(u_diff, v_diff)
    denominator = np.sqrt(sum(u_diff ** 2)) * np.sqrt(sum(v_diff ** 2))
    if denominator == 0:
        return 0.0
    return numerator / denominator


In [4]:
from collections import defaultdict

# 평갓값을 사용자 x 화면의 행렬로 변환한다
user_movie_matrix = movielens_train.pivot(index="user_id", columns="movie_id", values="rating")
user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))

# 예측 대상 사용자와 영화의 조합
movie_rating_predict = movielens_test.copy()
pred_user2items = defaultdict(list)

In [5]:
# 우직하게 유사도를 계산한다(이 계산은 매우 무겁습니다. 그렇기 떄문에 도중에 break를 넣었습니다. 속도 향상을 위해 라이브러리르 사용한 구현도 다음 셀에서 소개합니다)

# 예측 대상 사용자 ID
test_users = movie_rating_predict.user_id.unique()

# 예측 대상 사용자(사용자 1)에 주목한다
for user1_id in test_users:
    similar_users = []
    similarities = []
    avgs = []

    # 사용자 1과 평갓값 행렬 안의 기타 사용자(사용자 2)와의 유사도를 산출한다
    for user2_id in user_movie_matrix.index:
        if user1_id == user2_id:
            continue

        # 사용자 1과 사용자 2의 평갓값 벡터
        u_1 = user_movie_matrix.loc[user1_id, :].to_numpy()
        u_2 = user_movie_matrix.loc[user2_id, :].to_numpy()

        # `u_1`과 `u_2`로부터, 동시에 결손값이 없는 요소만 추출한 벡터를 얻는다
        common_items = ~np.isnan(u_1) & ~np.isnan(u_2)

        # 공통으로 평가한 아이템이 없으면 스킵
        if not common_items.any():
            continue

        u_1, u_2 = u_1[common_items], u_2[common_items]

        # 피어슨 상관 계수를 사용해 사용자 1과 사용자 2의 유사도를 산출
        rho_12 = peason_coefficient(u_1, u_2)

        # 사용자 1과의 유사도가 0 보다 큰 경우, 사용자 2를 유사 사용자로 간주한다
        if rho_12 > 0:
            similar_users.append(user2_id)
            similarities.append(rho_12)
            avgs.append(np.mean(u_2))

    # 사용자 1의 평균 평갓값
    avg_1 = np.mean(user_movie_matrix.loc[user1_id, :].dropna().to_numpy())

    # 예측 대상 영화의 ID
    test_movies = movie_rating_predict[movie_rating_predict["user_id"] == user1_id].movie_id.values
    # 예측할 수 없는 영화에 대한 평갓값은 사용자 1의 평균 평갓값으로 한다
    movie_rating_predict.loc[(movie_rating_predict["user_id"] == user1_id), "rating_pred"] = avg_1

    if similar_users:
        for movie_id in test_movies:
            if movie_id in movie_id2index:
                r_xy = user_movie_matrix.loc[similar_users, movie_id].to_numpy()
                rating_exists = ~np.isnan(r_xy)

                # 유사 사용자가 대상이 되는 영화에 대한 평갓값을 갖지 않은 경우는 스킵한다
                if not rating_exists.any():
                    continue

                r_xy = r_xy[rating_exists]
                rho_1x = np.array(similarities)[rating_exists]
                avg_x = np.array(avgs)[rating_exists]
                r_hat_1y = avg_1 + np.dot(rho_1x, (r_xy - avg_x)) / rho_1x.sum()

                # 예측 평갓값을 저장
                movie_rating_predict.loc[
                    (movie_rating_predict["user_id"] == user1_id)
                    & (movie_rating_predict["movie_id"] == movie_id),
                    "rating_pred",
                ] = r_hat_1y
    break # 계산이 무거우므로, for 루프는 1번만 수행한 뒤 종료합니다. 각 변수에 어떤 값이 들어있는지 확인하면 알고리즘을 더 깊이 이해할 수 있습니다.

In [6]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 771 kB 33.9 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626503 sha256=0e6db95d2a4c33d1ddb4f8a1bdfe526bfa9ce95d16c0f4728ccec069f280cbd1
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [8]:
# surprise를 사용한 구현

from surprise import KNNWithMeans, Reader
from surprise import Dataset as SurpriseDataset
# surprise용으로 데이터를 가공한다
reader = Reader(rating_scale=(0.5, 5))
data_train = SurpriseDataset.load_from_df(
    movielens_train[["user_id", "movie_id", "rating"]], reader
).build_full_trainset()

sim_options = {"name": "pearson", "user_based": True}  # 유사도를 계산하는 방법을 정의한다  # False로 하면 아이템 기반이 된다
knn = KNNWithMeans(k=30, min_k=1, sim_options=sim_options)
knn.fit(data_train)

# 학습 데이터셋에서 평갓값이 없는 사용자와 아이템의 조합을 준비한다
data_test = data_train.build_anti_testset(None)
predictions = knn.test(data_test)

def get_top_n(predictions, n=10):
    # 각 사용자별로 예측된 아이템을 저장한다
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # 사용자별로 아이템을 예측 평갓값 순으로 나열하고, 상위 n개를 저장한다
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [d[0] for d in user_ratings[:n]]

    return top_n

pred_user2items = get_top_n(predictions, n=10)

average_score = movielens_train.rating.mean()
pred_results = []
for _, row in movielens_test.iterrows():
    user_id = row["user_id"]
    movie_id = row["movie_id"]
    # 학습 데이터에 존재하지 않고 테스트 데이터에만 존재하는 사용자와 영화에 대한 예측 평갓값은 전체의 평균 평갓값으로 한다
    if user_id not in user_id2index or movie_id not in movie_id2index:
        pred_results.append(average_score)
        continue
    pred_score = knn.predict(uid=user_id, iid=movie_id).est
    pred_results.append(pred_score)
movie_rating_predict["rating_pred"] = pred_results

pred_user2items

Computing the pearson similarity matrix...
Done computing similarity matrix.


defaultdict(list,
            {139: [6319, 2721, 2131, 3415, 1121, 2509, 4221, 4418, 4509, 5101],
             149: [2494,
              26425,
              137,
              2930,
              1477,
              3473,
              27255,
              2911,
              2425,
              31116],
             182: [6319, 3415, 3473, 146, 1121, 2509, 4221, 4418, 4509, 6122],
             215: [1564, 3365, 1204, 1131, 2931, 2938, 1192, 1489, 2679, 1147],
             281: [137,
              1477,
              26425,
              6319,
              27255,
              2911,
              4850,
              5069,
              2494,
              31116],
             326: [1192, 2679, 3341, 7064, 32316, 3415, 3473, 4208, 4453, 146],
             351: [3473,
              6122,
              2911,
              137,
              27255,
              3777,
              1477,
              2930,
              26425,
              31116],
             357: [7064,
              

In [9]:
# user_id=2인 사용자가 학습 데이터에서 평가를 부여한 영화 목록
movielens_train[movielens_train.user_id==2]

user_id  movie_id  rating  timestamp  \
4732        2       110     5.0  868245777   
5246        2       260     5.0  868244562   
5798        2       590     5.0  868245608   
6150        2       648     2.0  868244699   
6531        2       733     3.0  868244562   
6813        2       736     3.0  868244698   
7113        2       780     3.0  868244698   
7506        2       786     3.0  868244562   
7661        2       802     2.0  868244603   
7779        2       858     2.0  868245645   
8077        2      1049     3.0  868245920   
8127        2      1073     3.0  868244562   
8381        2      1210     4.0  868245644   
8771        2      1356     3.0  868244603   
9097        2      1544     3.0  868245920   

                                                  title  \
4732                                  Braveheart (1995)   
5246  Star Wars: Episode IV - A New Hope (a.k.a. Sta...   
5798                          Dances with Wolves (1990)   
6150                         Mission: Impossible (1996)   
6531                                   Rock, The (1996)   
6813                                     Twister (1996)   
7113               Independence Day (a.k.a. ID4) (1996)   
7506                                      Eraser (1996)   
7661                                  Phenomenon (1996)   
7779                              Godfather, The (1972)   
8077                 Ghost and the Darkness, The (1996)   
8127         Willy Wonka & the Chocolate Factory (1971)   
8381  Star Wars: Episode VI - Return of the Jedi (1983)   
8771                    Star Trek: First Contact (1996)   
9097  Lost World: Jurassic Park, The (Jurassic Park ...   

                                              genre  \
4732                           [Action, Drama, War]   
5246                    [Action, Adventure, Sci-Fi]   
5798                    [Adventure, Drama, Western]   
6150         [Action, Adventure, Mystery, Thriller]   
6531                  [Action, Adventure, Thriller]   
6813         [Action, Adventure, Romance, Thriller]   
7113               [Action, Adventure, Sci-Fi, War]   
7506                      [Action, Drama, Thriller]   
7661                               [Drama, Romance]   
7779                                 [Crime, Drama]   
8077                            [Action, Adventure]   
8127           [Children, Comedy, Fantasy, Musical]   
8381                    [Action, Adventure, Sci-Fi]   
8771          [Action, Adventure, Sci-Fi, Thriller]   
9097  [Action, Adventure, Horror, Sci-Fi, Thriller]   

                                                    tag  timestamp_rank  
4732  [bullshit history, medieval, bloodshed, hero, ...             8.0  
5246  [desert, quotable, lucas, gfei own it, seen mo...            17.0  
5798  [afi 100, lame, native, biopic, american india...            11.0  
6150  [confusing, confusing plot, memorable sequence...            12.0  
6531  [gfei own it, alcatraz, nicolas cage, sean con...            18.0  
6813  [disaster, disaster, storm, bill paxton, helen...            13.0  
7113  [action, alien invasion, aliens, will smith, a...            14.0  
7506  [arnold schwarzenegger, action, arnold, arnold...            19.0  
7661  [interesting concept, own, john travolta, john...            15.0  
7779  [oscar (best picture), marlon brando, classic,...             9.0  
8077  [lions, own, seen at the cinema, adventure, af...             6.0  
8127  [based on a book, roald dahl, based on a book,...            20.0  
8381  [desert, fantasy, sci-fi, space, lucas, gfei o...            10.0  
8771  [space, borg, futuristmovies.com, patrick stew...            16.0  
9097  [dinosaurs, dinosaurs, steven spielberg, borin...             7.0

In [10]:
pred_user2items[2]

[3473, 4135, 4850, 2425, 2704, 137, 27255, 26425, 8712, 31116]

In [11]:
# user_id=2에 대한 추천(1198, 1196, 1097)
movies[movies.movie_id.isin([3473, 4135, 4850])]

movie_id                                              title  \
3386      3473  Jonah Who Will Be 25 in the Year 2000 (Jonas q...   
4043      4135                          Monster Squad, The (1987)   
4756      4850                        Spriggan (Supurigan) (1998)   

                            genre  \
3386                     [Comedy]   
4043  [Adventure, Comedy, Horror]   
4756  [Action, Animation, Sci-Fi]   

                                                    tag  
3386  [cerebral, humorous, intersecting lives, intim...  
4043                                   [can't remember]  
4756       [library, anime, spectacle, over expositive]